# K-Means Clustering with Scikit-Learn

In the previous lesson, we learned about a text analysis method called *term frequency–inverse document frequency*, often abbreviated *tf-idf*. Tf-idf is a method that tries to identify the most distinctively frequent or significant words in a document. We specifically learned how to calculate tf-idf scores using word frequencies per page—or "extracted features"—made available by the HathiTrust Digital Library.

In this lesson, we're going to learn how to calculate tf-idf scores using a collection of plain text (.txt) files and the Python library scikit-learn, which has a quick and nifty module called [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In this lesson, we will cover how to:
- Calculate and normalize tf-idf scores for U.S. Inaugural Addresses with scikit-learn

## Dataset

### U.S. Inaugural Addresses

```{epigraph}
This is the meaning of our liberty and our creed; why men and women and children of every race and every faith can join in celebration across this magnificent Mall, and why a man whose father less than 60 years ago might not have been served at a local restaurant can now stand before you to take a most sacred oath.  So let us mark this day with remembrance of who we are and how far we have traveled.

--  Barack Obama, Inaugural Presidential Address, January 2009 
```

During Barack Obama's Inaugural Address in January 2009, he mentioned "women" four different times, including in the passage quoted above. How distinctive is Obama's inclusion of women in this address compared to all other U.S. Presidents? This is one of the questions that we're going to try to answer with tf-idf.

## Breaking Down the TF-IDF Formula

But first, let's quickly discuss the tf-idf formula. The idea is pretty simple.

**tf-idf = term_frequency * inverse_document_frequency**

**term_frequency** = number of times a given term appears in document

**inverse_document_frequency** = log(total number of documents / number of documents with term) + 1**\***

You take the number of times a term occurs in a document (term frequency). Then you take the number of documents in which the same term occurs at least once divided by the total number of documents (document frequency), and you flip that fraction on its head (inverse document frequency). Then you multiply the two numbers together (term_frequency * inverse_document_frequency).

The reason we take the *inverse*, or flipped fraction, of document frequency is to boost the rarer words that occur in relatively few documents. Think about the inverse document frequency for the word "said" vs the word "pigeon." The term "said" appears in 13 (document frequency) of 14 (total documents) *Lost in the City* stories (14 / 13 --> a smaller inverse document frequency) while the term "pigeons" only occurs in 2 (document frequency) of the 14 stories (total documents) (14 / 2 --> a bigger inverse document frequency, a bigger tf-idf boost). 

*There are a bunch of slightly different ways that you can calculate inverse document frequency. The version of idf that we're going to use is the [scikit-learn default](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer), which uses "smoothing" aka it adds a "1" to the numerator and denominator: 

**inverse_document_frequency**  = log((1 + total_number_of_documents) / (number_of_documents_with_term +1)) + 1

```{margin}
> If smooth_idf=True (the default), the constant “1” is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions: idf(t) = log [ (1 + n) / (1 + df(t)) ] + 1.  
> -[scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)
```

## TF-IDF with scikit-learn

[scikit-learn](https://scikit-learn.org/stable/index.html), imported as `sklearn`, is a popular Python library for machine learning approaches such as clustering, classification, and regression. Though we're not doing any machine learning in this lesson, we're nevertheless going to use scikit-learn's `TfidfVectorizer` and `CountVectorizer`.

Install scikit-learn

In [45]:
!pip install sklearn

Import necessary modules and libraries

In [31]:
import altair as alt

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
pd.set_option("max_rows", 600)
from pathlib import Path  
import glob

We're also going to import `pandas` and change its default display setting. And we're going to import two libraries that will help us work with files and the file system: [`pathlib`](https://docs.python.org/3/library/pathlib.html##basic-use) and [`glob`](https://docs.python.org/3/library/glob.html).

#### Set Directory Path

Below we're setting the directory filepath that contains all the text files that we want to analyze.

In [4]:
directory_path = "../texts/history/US_Inaugural_Addresses/"

Then we're going to use `glob` and `Path` to make a list of all the filepaths in that directory and a list of all the short story titles.

In [5]:
text_files = glob.glob(f"{directory_path}/*.txt")

In [6]:
text_files

['../texts/history/US_Inaugural_Addresses/01_washington_1789.txt',
 '../texts/history/US_Inaugural_Addresses/02_washington_1793.txt',
 '../texts/history/US_Inaugural_Addresses/03_adams_john_1797.txt',
 '../texts/history/US_Inaugural_Addresses/04_jefferson_1801.txt',
 '../texts/history/US_Inaugural_Addresses/05_jefferson_1805.txt',
 '../texts/history/US_Inaugural_Addresses/06_madison_1809.txt',
 '../texts/history/US_Inaugural_Addresses/07_madison_1813.txt',
 '../texts/history/US_Inaugural_Addresses/08_monroe_1817.txt',
 '../texts/history/US_Inaugural_Addresses/09_monroe_1821.txt',
 '../texts/history/US_Inaugural_Addresses/10_adams_john_quincy_1825.txt',
 '../texts/history/US_Inaugural_Addresses/11_jackson_1829.txt',
 '../texts/history/US_Inaugural_Addresses/12_jackson_1833.txt',
 '../texts/history/US_Inaugural_Addresses/13_van_buren_1837.txt',
 '../texts/history/US_Inaugural_Addresses/14_harrison_1841.txt',
 '../texts/history/US_Inaugural_Addresses/15_polk_1845.txt',
 '../texts/history/

In [7]:
text_titles = [Path(text).stem for text in text_files]

In [8]:
text_titles

['01_washington_1789',
 '02_washington_1793',
 '03_adams_john_1797',
 '04_jefferson_1801',
 '05_jefferson_1805',
 '06_madison_1809',
 '07_madison_1813',
 '08_monroe_1817',
 '09_monroe_1821',
 '10_adams_john_quincy_1825',
 '11_jackson_1829',
 '12_jackson_1833',
 '13_van_buren_1837',
 '14_harrison_1841',
 '15_polk_1845',
 '16_taylor_1849',
 '17_pierce_1853',
 '18_buchanan_1857',
 '19_lincoln_1861',
 '20_lincoln_1865',
 '21_grant_1869',
 '22_grant_1873',
 '23_hayes_1877',
 '24_garfield_1881',
 '25_cleveland_1885',
 '26_harrison_1889',
 '27_cleveland_1893',
 '28_mckinley_1897',
 '29_mckinley_1901',
 '30_roosevelt_theodore_1905',
 '31_taft_1909',
 '32_wilson_1913',
 '33_wilson_1917',
 '34_harding_1921',
 '35_coolidge_1925',
 '36_hoover_1929',
 '37_roosevelt_franklin_1933',
 '38_roosevelt_franklin_1937',
 '39_roosevelt_franklin_1941',
 '40_roosevelt_franklin_1945',
 '41_truman_1949',
 '42_eisenhower_1953',
 '43_eisenhower_1957',
 '44_kennedy_1961',
 '45_johnson_1965',
 '46_nixon_1969',
 '47_

## Calculate tf–idf

To calculate tf–idf scores for every word, we're going to use scikit-learn's [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

When you initialize TfidfVectorizer, you can choose to set it with different parameters. These parameters will change the way you calculate tf–idf.

The recommended way to run `TfidfVectorizer` is with smoothing (`smooth_idf = True`) and normalization (`norm='l2'`) turned on. These parameters will better account for differences in text length, and overall produce more meaningful tf–idf scores. Smoothing and L2 normalization are actually the default settings for `TfidfVectorizer`, so to turn them on, you don't need to include any extra code at all.

Initialize TfidfVectorizer with desired parameters (default smoothing and normalization)

In [9]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english')

Run TfidfVectorizer on our `text_files`

In [10]:
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)

Make a DataFrame out of the resulting tf–idf vector, setting the "feature names" or words as columns and the titles as rows

In [11]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names())

Add column for document frequency aka number of times word appears in all documents

## K Means

In [12]:
from sklearn.cluster import KMeans

num_clusters = 4

km = KMeans(n_clusters=num_clusters, n_init=10) # default is also 10, but good to know 

km.fit(tfidf_vector)

# km.labels_ gives you the cluster assignments
clusters = km.labels_.tolist()

In [13]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
words = tfidf_vectorizer.get_feature_names()

top_words_list = []
for number in range(num_clusters):
    
    top_ten_words = [words[index] for index in order_centroids[number, :10]]
    top_words_list.append(" ".join(top_ten_words))
    print(f"---\nCluster {number}:\n{top_ten_words}")

---
Cluster 0:
['arrive', 'upbraidings', '1793', 'incurring', 'willingly', 'violated', 'knowingly', 'witnesses', 'previous', 'injunctions']
---
Cluster 1:
['world', 'people', 'peace', 'shall', 'war', 'nation', 'men', 'government', 'democracy', 'life']
---
Cluster 2:
['government', 'people', 'states', 'public', 'country', 'shall', 'constitution', 'union', 'great', 'power']
---
Cluster 3:
['america', 'world', 'new', 'nation', 'let', 'people', 'today', 'freedom', 'americans', 'time']


In [14]:
results = pd.DataFrame()
results['text'] = text_titles
results['category'] = km.labels_

In [15]:
results

,text,category
0,01_washington_1789,2
1,02_washington_1793,0
2,03_adams_john_1797,2
3,04_jefferson_1801,2
4,05_jefferson_1805,2
5,06_madison_1809,2
6,07_madison_1813,1
7,08_monroe_1817,2
8,09_monroe_1821,2
9,10_adams_john_quincy_1825,2


In [16]:
X = tfidf_vector.todense()

## PCA Plot

In [20]:
from sklearn.decomposition import PCA

In [21]:
pca_num_components = 2
reduced_data = PCA(n_components=pca_num_components).fit_transform(X)
# print reduced_data

In [22]:
reduced_df = pd.DataFrame(reduced_data)

In [23]:
reduced_df['title'] = text_titles

In [24]:
reduced_df['cluster'] = clusters

In [25]:
reduced_df = reduced_df.rename(columns= {0: 'x', 1: 'y'})

In [27]:
#Make a list of corresponding top words for each cluster to append to dataframe
top_words_column = []
number_of_clusters = reduced_df['cluster'].nunique()

for cluster in reduced_df['cluster']:
    for number in range(0, number_of_clusters):
        if cluster == number:
            top_words_column.append(top_words_list[number])

In [28]:
reduced_df['top_words'] = top_words_column

In [29]:
reduced_df

,x,y,title,cluster,top_words
0,-0.184639,-0.208635,01_washington_1789,2,government people states public country shall ...
1,-0.057761,-0.288762,02_washington_1793,0,arrive upbraidings 1793 incurring willingly vi...
2,-0.198036,-0.055405,03_adams_john_1797,2,government people states public country shall ...
3,-0.100516,-0.194398,04_jefferson_1801,2,government people states public country shall ...
4,-0.184482,-0.208315,05_jefferson_1805,2,government people states public country shall ...
5,-0.177750,-0.284071,06_madison_1809,2,government people states public country shall ...
6,-0.091372,-0.244205,07_madison_1813,1,world people peace shall war nation men govern...
7,-0.333489,-0.057452,08_monroe_1817,2,government people states public country shall ...
8,-0.279557,-0.051897,09_monroe_1821,2,government people states public country shall ...
9,-0.279912,-0.135633,10_adams_john_quincy_1825,2,government people states public country shall ...


In [32]:
alt.Chart(reduced_df).mark_circle(size=60).encode(
    x='x',
    y='y',
    color=alt.Color('cluster', scale=alt. 
                      Scale(scheme = 'dark2'), type='ordinal') ,
    tooltip=['title', 'cluster', 'top_words']
).interactive()

alt.Chart(...)

## TSNE Plot

In [218]:
tsne_num_components = 2
embeddings = TSNE(n_components=tsne_num_components)
Y = embeddings.fit_transform(X)

In [219]:
tsne_df = pd.DataFrame(Y)

In [220]:
tsne_df['title'] = text_titles

In [221]:
tsne_df['cluster'] = clusters

In [222]:
tsne_df = tsne_df.rename(columns= {0: 'x', 1: 'y'})

In [223]:
#Make a list of corresponding top words for each cluster to append to dataframe
top_words_column = []
number_of_clusters = reduced_df['cluster'].nunique()

for cluster in tsne_df['cluster']:
    for number in range(0, number_of_clusters):
        if cluster == number:
            top_words_column.append(top_words_list[number])

In [224]:
tsne_df['top_words'] = top_words_column

In [228]:
alt.Chart(tsne_df).mark_circle(size=60).encode(
    x='x',
    y='y',
    color=alt.Color('cluster', scale=alt. 
                      Scale(scheme = 'dark2'),type='ordinal') ,
    tooltip=['title', 'cluster', 'top_words']
).interactive()

alt.Chart(...)

## K Means — NYT Obituaries

In [229]:
directory_path = "../texts/history/NYT-Obituaries/"

Then we're going to use `glob` and `Path` to make a list of all the filepaths in that directory and a list of all the short story titles.

In [230]:
text_files = glob.glob(f"{directory_path}/*.txt")

In [231]:
text_files

['../texts/history/NYT-Obituaries/1852-Ada-Lovelace.txt',
 '../texts/history/NYT-Obituaries/1870-Robert-E-Lee.txt',
 '../texts/history/NYT-Obituaries/1875-Andrew-Johnson.txt',
 '../texts/history/NYT-Obituaries/1877-Bedford-Forrest.txt',
 '../texts/history/NYT-Obituaries/1880-Lucretia-Mott.txt',
 '../texts/history/NYT-Obituaries/1882-Charles-Darwin.txt',
 '../texts/history/NYT-Obituaries/1885-Ulysses-Grant.txt',
 '../texts/history/NYT-Obituaries/1886-Mary-Ewing-Outerbridge.txt',
 '../texts/history/NYT-Obituaries/1887-Emma-Lazarus.txt',
 '../texts/history/NYT-Obituaries/1888-Louisa-M-Alcott.txt',
 '../texts/history/NYT-Obituaries/1891-P-T-Barnum.txt',
 '../texts/history/NYT-Obituaries/1894-R-L-Stevenson.txt',
 '../texts/history/NYT-Obituaries/1895-Fred-Douglass.txt',
 '../texts/history/NYT-Obituaries/1896-Harriet-Beecher-Stowe.txt',
 '../texts/history/NYT-Obituaries/1900-Nietzsche.txt',
 '../texts/history/NYT-Obituaries/1900-Stephen-Crane.txt',
 '../texts/history/NYT-Obituaries/1901-Benj

In [232]:
text_titles = [Path(text).stem for text in text_files]

In [233]:
text_titles

['1852-Ada-Lovelace',
 '1870-Robert-E-Lee',
 '1875-Andrew-Johnson',
 '1877-Bedford-Forrest',
 '1880-Lucretia-Mott',
 '1882-Charles-Darwin',
 '1885-Ulysses-Grant',
 '1886-Mary-Ewing-Outerbridge',
 '1887-Emma-Lazarus',
 '1888-Louisa-M-Alcott',
 '1891-P-T-Barnum',
 '1894-R-L-Stevenson',
 '1895-Fred-Douglass',
 '1896-Harriet-Beecher-Stowe',
 '1900-Nietzsche',
 '1900-Stephen-Crane',
 '1901-Benjamin-Harrison',
 '1901-Queen-Victoria',
 '1901-William-McKinley',
 '1902-Elizabeth-Cady-Stanton',
 '1903-Emily-Warren-Roebling',
 '1903-James-M-N-Whistler',
 '1906-Susan-B-Anthony',
 '1907-Qiu-Jin',
 '1908-Cleveland',
 '1909-Geronimo',
 '1909-Sarah-Orne-Jewett',
 '1910-Florence-Nightingale',
 '1910-Tolstoy',
 '1910-William-James',
 '1911-Joseph-Pulitzer',
 '1914-Alfred-Thayer-Mahan',
 '1914-John-Muir',
 '1914-John-P-Holland',
 '1915-B-T-Washington',
 '1915-F-W-Taylor',
 '1916-J-J-Hill',
 '1916-Jack-London',
 '1916-Martian-Theory',
 '1917-Hilaire-G-E-Degas',
 '1919-Anna-H-Shaw',
 '1919-C-J-Walker',
 '1

## Calculate tf–idf

To calculate tf–idf scores for every word, we're going to use scikit-learn's [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

When you initialize TfidfVectorizer, you can choose to set it with different parameters. These parameters will change the way you calculate tf–idf.

The recommended way to run `TfidfVectorizer` is with smoothing (`smooth_idf = True`) and normalization (`norm='l2'`) turned on. These parameters will better account for differences in text length, and overall produce more meaningful tf–idf scores. Smoothing and L2 normalization are actually the default settings for `TfidfVectorizer`, so to turn them on, you don't need to include any extra code at all.

Initialize TfidfVectorizer with desired parameters (default smoothing and normalization)

In [234]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english')

Run TfidfVectorizer on our `text_files`

In [235]:
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)

Make a DataFrame out of the resulting tf–idf vector, setting the "feature names" or words as columns and the titles as rows

In [261]:
tfidf_df

,00,000,000f,001,006,01,010,021,025,028,...,zrathustra,zuber,zuker,zukor,zukors,zula,zululand,zurich,zvai,zwilich
1852-Ada-Lovelace,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1870-Robert-E-Lee,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1875-Andrew-Johnson,0.000000,0.007602,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1877-Bedford-Forrest,0.000000,0.018923,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1880-Lucretia-Mott,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1882-Charles-Darwin,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1885-Ulysses-Grant,0.000000,0.053234,0.000000,0.00351,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1886-Mary-Ewing-Outerbridge,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1887-Emma-Lazarus,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1888-Louisa-M-Alcott,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [237]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names())

Add column for document frequency aka number of times word appears in all documents

In [262]:
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters, n_init=10) # default is also 10, but good to know 

km.fit(tfidf_vector)

# km.labels_ gives you the cluster assignments
clusters = km.labels_.tolist()

In [263]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
words = tfidf_vectorizer.get_feature_names()

top_words_list = []
for number in range(num_clusters):
    
    top_ten_words = [words[index] for index in order_centroids[number, :10]]
    top_words_list.append(" ".join(top_ten_words))
    print(f"---\nCluster {number}:\n{top_ten_words}")

---
Cluster 0:
['tennis', 'game', 'ashe', 'runs', 'rockne', 'wills', 'outerbridge', 'golf', 'abbott', 'jones']
---
Cluster 1:
['mr', 'israel', 'soviet', 'sadat', 'jewish', 'pulitzer', 'brezhnev', 'buber', 'said', 'borges']
---
Cluster 2:
['years', 'war', 'time', 'new', 'said', 'life', '000', 'world', 'general', 'work']
---
Cluster 3:
['mr', 'cleveland', 'baseball', 'douglass', 'hemingway', 'rockefeller', 'thorpe', 'catton', 'ochs', 'ruth']
---
Cluster 4:
['sullivan', 'keller', 'murrow', 'mr', 'dempsey', 'mrs', 'miss', 'durocher', 'zaharias', 'macy']
---
Cluster 5:
['mr', 'jazz', 'band', 'music', 'blues', 'armstrong', 'piano', 'basie', 'goodman', 'gillespie']
---
Cluster 6:
['miss', 'jackson', 'years', 'said', 'new', 'ziegfeld', 'jewett', 'hellman', 'balch', 'graham']
---
Cluster 7:
['dr', 'dewey', 'university', 'research', 'negro', 'institute', 'vaccine', 'atomic', 'professor', 'jung']
---
Cluster 8:
['mr', 'president', 'roosevelt', 'said', 'years', 'kennedy', 'hoover', 'mrs', 'state',

In [264]:
results = pd.DataFrame()
results['text'] = text_titles
results['category'] = km.labels_

In [265]:
results

,text,category
0,1852-Ada-Lovelace,2
1,1870-Robert-E-Lee,2
2,1875-Andrew-Johnson,8
3,1877-Bedford-Forrest,2
4,1880-Lucretia-Mott,8
5,1882-Charles-Darwin,2
6,1885-Ulysses-Grant,2
7,1886-Mary-Ewing-Outerbridge,0
8,1887-Emma-Lazarus,2
9,1888-Louisa-M-Alcott,6


In [266]:
X = tfidf_vector.todense()

## PCA Plot

In [267]:
pca_num_components = 2
reduced_data = PCA(n_components=pca_num_components).fit_transform(X)
# print reduced_data

In [268]:
reduced_df = pd.DataFrame(reduced_data)

In [269]:
reduced_df['title'] = text_titles

In [270]:
reduced_df['cluster'] = clusters

In [271]:
reduced_df = reduced_df.rename(columns= {0: 'x', 1: 'y'})

In [272]:
#Make a list of corresponding top words for each cluster to append to dataframe
top_words_column = []
number_of_clusters = reduced_df['cluster'].nunique()

for cluster in reduced_df['cluster']:
    for number in range(0, number_of_clusters):
        if cluster == number:
            top_words_column.append(top_words_list[number])

In [273]:
reduced_df['top_words'] = top_words_column

In [274]:
reduced_df

,x,y,title,cluster,top_words
0,-0.097039,-0.024191,1852-Ada-Lovelace,2,years war time new said life 000 world general...
1,0.030303,-0.145161,1870-Robert-E-Lee,2,years war time new said life 000 world general...
2,0.230276,-0.164679,1875-Andrew-Johnson,8,mr president roosevelt said years kennedy hoov...
3,-0.006883,-0.087962,1877-Bedford-Forrest,2,years war time new said life 000 world general...
4,-0.062079,-0.041888,1880-Lucretia-Mott,8,mr president roosevelt said years kennedy hoov...
5,-0.036459,-0.047472,1882-Charles-Darwin,2,years war time new said life 000 world general...
6,0.127220,-0.208108,1885-Ulysses-Grant,2,years war time new said life 000 world general...
7,-0.061829,-0.028920,1886-Mary-Ewing-Outerbridge,0,tennis game ashe runs rockne wills outerbridge...
8,-0.077708,-0.032835,1887-Emma-Lazarus,2,years war time new said life 000 world general...
9,-0.129666,-0.021390,1888-Louisa-M-Alcott,6,miss jackson years said new ziegfeld jewett he...


In [275]:
alt.Chart(reduced_df).mark_circle(size=60).encode(
    x='x',
    y='y',
    color=alt.Color('cluster', scale=alt. 
                      Scale(scheme = 'dark2'), type='ordinal') ,
    tooltip=['title', 'cluster', 'top_words']
).interactive()

alt.Chart(...)

## TSNE Plot

In [276]:
tsne_num_components = 2
embeddings = TSNE(n_components=tsne_num_components)
Y = embeddings.fit_transform(X)

In [277]:
tsne_df = pd.DataFrame(Y)

In [278]:
tsne_df['title'] = text_titles

In [279]:
tsne_df['cluster'] = clusters

In [280]:
tsne_df = tsne_df.rename(columns= {0: 'x', 1: 'y'})

In [281]:
#Make a list of corresponding top words for each cluster to append to dataframe
top_words_column = []
number_of_clusters = reduced_df['cluster'].nunique()

for cluster in tsne_df['cluster']:
    for number in range(0, number_of_clusters):
        if cluster == number:
            top_words_column.append(top_words_list[number])

In [282]:
tsne_df['top_words'] = top_words_column

In [283]:
alt.Chart(tsne_df).mark_circle(size=60).encode(
    x='x',
    y='y',
    color=alt.Color('cluster', scale=alt. 
                      Scale(scheme = 'dark2'),type='ordinal') ,
    tooltip=['title', 'cluster', 'top_words']
).interactive()

alt.Chart(...)